## GENAI - 생성형 이미지로 텍스트 제거

In [18]:
from google import genai
from google.genai import types
from PIL import Image
from io import BytesIO
import os

import PIL.Image
from dotenv import load_dotenv

load_dotenv()



True

In [14]:
image = PIL.Image.open('test3.jpg')

client = genai.Client(api_key=os.getenv('GEMINI_API_KEY'))

text_input = ('translate all texts from image into english. Return the translated image')

response = client.models.generate_content(
    model="gemini-2.0-flash-exp-image-generation",
    contents=[text_input, image],
    config=types.GenerateContentConfig(
      response_modalities=['TEXT', 'IMAGE']
    )
)  

print(response)

for part in response.candidates[0].content.parts:
  if part.text is not None:
    print(part.text)
  elif part.inline_data is not None:
    image = Image.open(BytesIO(part.inline_data.data))
    image.show()

candidates=[Candidate(content=Content(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, inline_data=Blob(data=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x04\x00\x00\x00\x02\xa6\x08\x02\x00\x00\x00\\\xea{\x1d\x00\x00 \x00IDATx\xda\xec\xfd\x87\x93$g\x96\x1f\x08\xce\x7fs\xb45;#\xcfn\xf7\xb8\xb7\xb7\xc7\xbd%\x97\xcd\xe1p\x14g\xc9\x1d\x0e\xc9\xe9\x99i-\xd1\xba\x1b-\xd0@C6\xd0\xd0@CV\x01\x85\x82\xaaBi]\x95\x99\x95:t\x84{\xe8T\x95%R\x84v\xad\xf5\xbd\xf7ydf\xe8\xf0HQ\xa8\xea.\xb3ga\x91\x9e\x11\x1e.>\x7f\xef\xf7{\xf2O<)\xed\x894y\xcd\x10\xc9n\xbd\x19,i"\xdbo\x06K&\xc0\x07z\xfeD\xa6\xfd\xbb=\x0f\xa0\xe3\xf3\xe9>\xfb!\xaf\x02H\xfa\xf3\x96\xcc\x08"fF\xfb\xfc\x8e\xa4\xef\x8d\x93M\xb7\x1cIf\xd8\xf6\xcc\xc8\xd7g/"\x8e~\x85\xc5\x96o\x89\xed\x7f\xee\xe9f\xf5\xf9!\xffa\x14\xb2D\xfc\x7fe\x9b\xdb\x9b\xef\xb7\xb6\x8b[\xdbw\xa4\xe3\xcfl\xcb\xa3\xdd\xfa\xa6\xdf\x07\x1e\xc8\xbd$Bf\xe7\xbe\x8b\x1d\xf7\

## Clipdrop API - 자동 텍스트 제거

In [23]:
import requests

image_path = '테스트 이미지3(영문).png'

# 확장자 확인
ext = os.path.splitext(image_path)[1].lower()

if ext == '.webp':
    # webp 이미지를 열어서 png로 변환
    with Image.open(image_path) as im:
        buf = BytesIO()
        im.save(buf, format='PNG')
        buf.seek(0)
        image_file_object = buf
        filename = 'image.png'
        mime_type = 'image/png'
else:
    # webp가 아니면 원본 파일 그대로 사용
    image_file_object = open(image_path, 'rb')
    filename = os.path.basename(image_path)
    # 확장자에 따라 mime_type 지정
    mime_type = {
        '.png': 'image/png',
        '.jpg': 'image/jpeg',
        '.jpeg': 'image/jpeg',
        '.bmp': 'image/bmp'
    }.get(ext, 'application/octet-stream')

r = requests.post('https://clipdrop-api.co/remove-text/v1',
  files = {
    'image_file': ('image.webp', image_file_object, 'image/webp')
    },
  headers = { 'x-api-key': os.getenv('CLIPDROP_API_KEY')}
)
if (r.ok):
  # r.content contains the bytes of the returned image
  image_file_object = BytesIO(r.content)
  image = Image.open(image_file_object)
  image.show()
  image.save('test_clipdrop.webp')
else:
  r.raise_for_status()